In [ ]:
!pip install tifffile

In [ ]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import tifffile
import time
from datetime import datetime
import pandas as pd
pd.set_option('display.max_colwidth', 250)
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.geometry import shape
import glob
from shapely.geometry import box
import rasterio
from rasterio.windows import Window
import json
from matplotlib.colors import LinearSegmentedColormap
red_to_green = LinearSegmentedColormap.from_list("RedGreen", ["red", "green"])

from src.train import train
from src.hyperparams import HyperParams, open_from_yaml
from src.data import *
from src.validate import return_metrics_all_folds, visualize_single_model, cross_validation, visualize_s2_concat_bands_path
from src.visualize import visualize_s2_concat_bands_path, visualize_s1_path, scale_img, visualize_ls_concat_bands_path, visualize_cb_mux_path, visualize_cb_wpm_path

In [ ]:
sorted(glob.glob("trained_models/ExpS2*/*/*/best_metric*"))

In [ ]:
sorted(glob.glob("trained_models/ExpS1*/*/*/best_metric*"))

# S2 single sensor, single before/after image

In [ ]:
sorted(glob.glob("trained_models/ExpS2*/*/*/best_metric*"))

In [ ]:
hps_list = []
hps = HyperParams(**open_from_yaml("trained_models/ExpS2_single_2-0"))
hps_list.append(hps)
df = pd.read_pickle(hps.df_path)

pred_name = "_".join([hps_.name for hps_ in hps_list])
print(pred_name)
# threshold = 0.5 #0.8 #0.3

In [ ]:
also_save_preds = True
threshold = 0.5
fold_nbs = [0, 1, 2, 3, 4] #[0, 1] #, 2, 3, 4]#, 1, 2, 3, 4] #[4] 

subset = "val_all_folds"
export = return_metrics_all_folds(hps_list, threshold=threshold, on_gpu=True, output_spatial_size=(400, 400), also_save_preds=also_save_preds,
                                  save_path=f"/data/C2S_CNN/predictions/{pred_name}_{subset}.bc", fold_nbs=fold_nbs)
# export_path = f"data/metrics_{pred_name}_{subset}.npy"
# np.save(export_path, export)

In [ ]:
probs, losses, tps, fps, fns = export
probs.shape

In [ ]:
img_paths_before, img_paths_after, GT_paths = [], [], []
hps.only_val = True
for fold_nb in fold_nbs:    
    hps.fold_nb = fold_nb
    val_dataset, val_loader = get_dataloaders(hps=hps)
    img_paths_before.extend(val_dataset.img_paths_before)
    img_paths_after.extend(val_dataset.img_paths_after)
    GT_paths.extend(val_dataset.mask_paths)

print(len(img_paths_before), len(img_paths_after), len(GT_paths))

In [ ]:
selection_criteria = "FP+FN" #"FP+FN" #"FP+FN ratio" #"(FP+FN)/TP ratio" #"FP+FN" #"random" #"FP+FN" #"random" 

looked_at_paths, looked_at_idx = visualize_single_model(
    df, tps, fps, fns, [probs], threshold, img_paths_before, img_paths_after, GT_paths, export_how_many=50, skip_how_many=0, alpha=0.4, 
    selection_criteria=selection_criteria)

# S1 single sensor, single before/after image

In [ ]:
sorted(glob.glob("trained_models/*/*/*/best_metric*"))

In [ ]:
hps_list = []
hps = HyperParams(**open_from_yaml("trained_models/ExpS1_single_2-0"))
hps_list.append(hps)
df = pd.read_pickle(hps.df_path)

pred_name = "_".join([hps_.name for hps_ in hps_list])
print(pred_name)
# threshold = 0.5 #0.8 #0.3

In [ ]:
also_save_preds = True
threshold = 0.5
fold_nbs = [0, 1, 2, 3, 4]#, 1, 2, 3, 4] #[4] 

subset = "val_all_folds"
export = return_metrics_all_folds(hps_list, threshold=threshold, on_gpu=True, output_spatial_size=(400, 400), also_save_preds=also_save_preds,
                                  save_path=f"/data/C2S_CNN/predictions/{pred_name}_{subset}.bc", fold_nbs=fold_nbs)
# export_path = f"data/metrics_{pred_name}_{subset}.npy"
# np.save(export_path, export)

In [ ]:
probs, losses, tps, fps, fns = export
probs.shape

In [ ]:
img_paths_before, img_paths_after, GT_paths = [], [], []
hps.only_val = True
for fold_nb in fold_nbs:    
    hps.fold_nb = fold_nb
    val_dataset, val_loader = get_dataloaders(hps=hps)
    img_paths_before.extend(val_dataset.img_paths_before)
    img_paths_after.extend(val_dataset.img_paths_after)
    GT_paths.extend(val_dataset.mask_paths)

print(len(img_paths_before), len(img_paths_after), len(GT_paths))

In [ ]:
selection_criteria = "FP+FN" #"FP+FN" #"FP+FN ratio" #"(FP+FN)/TP ratio" #"FP+FN" #"random" #"FP+FN" #"random" 
specific_location = 0 #"Senanga" #"Matadi" #"South America - Argentina" #0 #"Santa Cruz do Sul" #"Timbuktu" #0 #"Eritrea" #0

looked_at_paths, looked_at_idx = visualize_single_model(
    df, tps, fps, fns, [probs], threshold, img_paths_before, img_paths_after, GT_paths, export_how_many=50, skip_how_many=0, alpha=0.5, 
    selection_criteria=selection_criteria)